# MLHC Training Pipeline

> End‑to‑end model development for ICU outcomes (mortality, prolonged length of stay, 30‑day readmission) starting from precomputed first‑48h features (`features_full.parquet`) and labels (`labels.csv`).


## Section Map
1. Environment Setup – paths, seed, versions.
2. Cohort & Admissions – load + strictly align features & labels; variance check.
3. (Legacy) Quick Split Preview – initial readmission‑stratified split (diagnostic only).
4. Feature / Metric Utilities – helper metric functions (F1 / cost / Fβ).
5. Target Specs & Canonical Split – define three targets & build one shared stratified train/val/test split.
6. Split Persistence & Baseline Characteristics – persist subject ID lists + wide Table 1.
7. Model Training & Tuning – Optuna XGBoost CV, final + base (calibration) models, isotonic calibration, threshold (max F1).
8. Metrics Aggregation & Artifact Index – consolidate per‑target metrics & artifact paths.
9. ROC & PR Curves (Calibrated) – discrimination visualizations.
10. Reliability (Calibration) Curves – probability calibration assessment.
11. Manual Calibration Example – illustrative standalone isotonic workflow.
12. Metrics Summary & Persistence – tuned vs baseline; full vs calibrated; exports.
13. Inference & Enriched Metrics – merged summary for downstream consumption.
14. SHAP Feature Attribution – global feature importance from calibrated base models.

### Outputs (Key Files in `project/artifacts/`)
- `model_{target}.joblib` + `isotonic_{target}.joblib` + `threshold_{target}.txt`
- `model_full_{target}.joblib` (train+val retrain)
- `metrics_{target}.json`, `metrics_all.json`, `metrics_summary*.csv`
- `train/val/test_subject_ids.txt`, `data_split_manifest.json`
- `table1_patient_characteristics.csv`, `table1_patient_characteristics_pivoted.csv`
- `full_vs_calibrated_comparison.csv`, `auc_diff_significance.csv` (if bootstrap completed)
- `shap_feature_importance_{target}.csv`, `shap_values_{target}.npy`, `shap_summary_{target}.png`

> Proceed sequentially; rerun Sections 7–15 after modifying features, labels, or hyperparameter search space.

In [ ]:
# Environment & core imports
import os, sys, json, random, platform, importlib, datetime, pathlib
from pathlib import Path
import numpy as np, pandas as pd

# Add project root to path if notebook is under notebooks/
ROOT = pathlib.Path(__file__).resolve().parents[1] if '__file__' in globals() else pathlib.Path.cwd().parents[0]
if str(ROOT) not in sys.path: sys.path.insert(0, str(ROOT))


SEED = 42
random.seed(SEED); np.random.seed(SEED)
PROJECT_ROOT = (Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd())
DATA_DIR = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'project/artifacts'
RUNS_ROOT = PROJECT_ROOT / 'runs'
print(f"Project root: {PROJECT_ROOT}")
print(f"Data dir exists: {(DATA_DIR).exists()}")
VERSIONS = {'python': sys.version.split()[0], 'platform': platform.platform()}
for pkg in ['xgboost','optuna','shap','sklearn','pandas','numpy']:
    try:
        m = importlib.import_module(pkg if pkg != 'sklearn' else 'sklearn')
        VERSIONS[pkg] = getattr(m,'__version__','?')
    except Exception as e:
        VERSIONS[pkg] = f'NA({e})'
print('Versions:', json.dumps(VERSIONS, indent=2))

## 1. Environment Setup
Configure paths, deterministic seed, and collect package versions for reproducibility.

In [ ]:
# Load canonical labels produced by duckdb_extraction notebook
import pandas as pd, random
from pathlib import Path

# Force single authoritative path (output of duckdb_extraction)
LABELS_PATH = (PROJECT_ROOT / 'project' / 'artifacts' / 'labels.csv')
if not LABELS_PATH.exists():
    raise FileNotFoundError(f'Expected labels at {LABELS_PATH}. Run duckdb_extraction notebook first to generate labels.csv there.')

labels_df = pd.read_csv(LABELS_PATH)
# Normalize required columns
required_cols = {'subject_id','hadm_id','readmission_label','mortality_label','prolonged_los_label'}
missing = required_cols - set(labels_df.columns)
if missing:
    raise ValueError(f'Missing expected label columns in {LABELS_PATH}: {missing}')

labels_df = labels_df.drop_duplicates('subject_id')
for col in ['readmission_label','mortality_label','prolonged_los_label']:
    labels_df[col] = labels_df[col].astype(int)
assert labels_df['subject_id'].isna().sum()==0
prev_readmit = labels_df['readmission_label'].mean()
print(f'Labels source: {LABELS_PATH} | shape={labels_df.shape} | readmission_prevalence={prev_readmit:.4f}')

## 2. Cohort & Admissions
Load pre‑built feature matrix; enforce index uniqueness and full subject overlap with labels (hard fail if any mismatch). Compute variance preservation check after realignment.

In [ ]:
# Load feature matrix (regenerate if tiny/corrupt) + strict safe alignment
import pandas as pd, numpy as np, json, os
from pathlib import Path

feature_path = ARTIFACTS_DIR / 'features_full.parquet'
if not feature_path.exists():
    raise FileNotFoundError(f'Missing feature parquet: {feature_path}')

feature_df = pd.read_parquet(feature_path)
regenerated = False  # placeholder if regeneration logic later added

# Ensure subject_id index
if 'subject_id' in feature_df.columns:
    if feature_df['subject_id'].duplicated().any():
        raise RuntimeError('Duplicate subject_id rows in feature parquet.')
    feature_df = feature_df.set_index('subject_id')

# Canonical integer index (fix KeyError mismatch from mixed string/int earlier)
try:
    feature_df.index = feature_df.index.astype(int)
except Exception as e:
    raise RuntimeError(f'Failed to coerce feature index to int: {e}')
if feature_df.index.has_duplicates:
    raise RuntimeError('Feature index has duplicates after int coercion.')

# Labels already loaded upstream into labels_df
label_subjects = labels_df['subject_id'].astype(int)

feat_ids = feature_df.index
missing_in_features = set(label_subjects) - set(feat_ids)
extra_in_features = set(feat_ids) - set(label_subjects)
overlap_frac = 1 - (len(missing_in_features)/len(label_subjects)) if len(label_subjects) else 0
print(f"[cohort-diff] label_subjects={len(label_subjects)} feature_subjects={len(feat_ids)}")
print(f"[cohort-diff] missing_in_features={len(missing_in_features)} extra_in_features={len(extra_in_features)}")
print(f"[align] overlap_frac={overlap_frac:.4f}")

if missing_in_features:
    raise RuntimeError(f'Missing {len(missing_in_features)} subjects (sample: {list(missing_in_features)[:10]})')

# Pre-alignment variance snapshot (for collapse detection)
pre_var = feature_df.var(numeric_only=True, ddof=0)

# Strict ordering alignment
feature_df_aligned = feature_df.loc[label_subjects.values]

post_var = feature_df_aligned.var(numeric_only=True, ddof=0)
collapsed = [c for c in pre_var.index if pre_var[c] > 0 and post_var[c] == 0]
if collapsed:
    print(f'WARNING: {len(collapsed)} columns lost variance post-alignment (e.g. {collapsed[:5]}).')
else:
    print('[align] Variance preserved.')

print('Features aligned shape:', feature_df_aligned.shape, '| regenerated' if regenerated else '')

# Expose for downstream cells
features_loaded = feature_df_aligned

## 3. Quick Split Preview
Perform an initial stratified split on readmission only (legacy diagnostic). Canonical multi‑target split is created later; this section is retained for prevalence sanity checks.

In [ ]:
# Train/valid/test split (60/20/20) + class weight factor
from sklearn.model_selection import train_test_split
readmit_y = labels_df['readmission_label'].astype(int).to_numpy()
subject_index = feature_df.index.to_numpy()
X = feature_df.values
X_tr, X_temp, y_tr, y_temp, sid_tr, sid_temp = train_test_split(
    X, readmit_y, subject_index, test_size=0.4, stratify=readmit_y, random_state=SEED)
X_val, X_te, y_val, y_te, sid_val, sid_te = train_test_split(
    X_temp, y_temp, sid_temp, test_size=0.5, stratify=y_temp, random_state=SEED)
pos_rate = y_tr.mean(); scale_pos_weight = (1-pos_rate)/max(pos_rate,1e-6)
print(f'Split -> train {X_tr.shape} valid {X_val.shape} test {X_te.shape} | pos_rate_train={pos_rate:.4f} | spw≈{scale_pos_weight:.2f}')

## 4. Feature / Metric Utilities
Utility functions (e.g., cost‑aware + F/FBeta metrics) used later for threshold selection and diagnostics.

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, confusion_matrix
import numpy as np
C_FP = 1.0; C_FN = 5.0
beta = 2.0

def metrics_at(proba, y, thr):
    pred = (proba >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    cost = C_FP*fp + C_FN*fn
    f1 = f1_score(y, pred)
    prec = tp/(tp+fp+1e-9); rec = tp/(tp+fn+1e-9)
    fbeta = (1+beta**2)*prec*rec/(beta**2*prec+rec+1e-9)
    return dict(f1=f1, precision=prec, recall=rec, cost=cost, fbeta=fbeta)

## 5. Target Specs & Canonical Split
Define the three binary targets and create a single shared train/val/test split (stratified on the first target) reused for all models to ensure comparable evaluation and fairness alignment.

In [ ]:

SEED = 42
rng_global = np.random.default_rng(SEED)

TARGET_SPECS = [
    ('readmission','readmission_label'),
    ('mortality','mortality_label'),
    ('prolonged_los','prolonged_los_label'),
]


# Verify presence of target columns
missing_targets = [lbl for _,lbl in TARGET_SPECS if lbl not in labels_df.columns]
if missing_targets:
    raise ValueError(f"Missing target label columns: {missing_targets}")

# Shared subject index + base split for comparability
subject_index = feature_df.index.to_numpy()
X = feature_df.values
primary_y = labels_df[TARGET_SPECS[0][1]].astype(int).to_numpy()  # stratify on first target
X_tr, X_temp, y_tr_primary, y_temp_primary, sid_tr, sid_temp = train_test_split(X, primary_y, subject_index, test_size=0.4, stratify=primary_y, random_state=SEED)
X_val, X_te, y_val_primary, y_te_primary, sid_val, sid_te = train_test_split(X_temp, y_temp_primary, sid_temp, test_size=0.5, stratify=y_temp_primary, random_state=SEED)
print(f'Shared split shapes -> train {X_tr.shape} val {X_val.shape} test {X_te.shape}')

subj_to_pos = {sid:i for i,sid in enumerate(subject_index)}
idx_tr = np.array([subj_to_pos[s] for s in sid_tr])
idx_val = np.array([subj_to_pos[s] for s in sid_val])
idx_te = np.array([subj_to_pos[s] for s in sid_te])
metrics_all = {}


In [ ]:
# Persist authoritative split subject IDs for downstream fairness / reproducibility
import json, time
ARTIFACTS_DIR.mkdir(exist_ok=True)

split_manifest = {
    'created_utc': time.time(),
    'seed': SEED,
    'counts': {
        'train': int(len(sid_tr)),
        'val': int(len(sid_val)),
        'test': int(len(sid_te)),
        'total': int(len(subject_index)),
    },
    'subject_ids': {
        'train': [int(x) for x in sid_tr.tolist()],
        'val': [int(x) for x in sid_val.tolist()],
        'test': [int(x) for x in sid_te.tolist()],
    },
}
# Lightweight plain‑text convenience files (one id per line)
with open(ARTIFACTS_DIR / 'train_subject_ids.txt','w') as f:
    f.write('\n'.join(str(int(x)) for x in sid_tr))
with open(ARTIFACTS_DIR / 'val_subject_ids.txt','w') as f:
    f.write('\n'.join(str(int(x)) for x in sid_val))
with open(ARTIFACTS_DIR / 'test_subject_ids.txt','w') as f:
    f.write('\n'.join(str(int(x)) for x in sid_te))
with open(ARTIFACTS_DIR / 'data_split_manifest.json','w') as f:
    json.dump(split_manifest, f, indent=2)
print('Persisted split subject id artifacts ->', (ARTIFACTS_DIR / 'data_split_manifest.json').resolve())

## 6. Split Persistence & Baseline Characteristics
Persist split subject IDs (JSON + txt) for downstream reproducibility and fairness. Then generate Table 1 (wide + pivot) summarizing demographics, outcomes, and key physiologic variables across splits.

#### 6b. Pivoted Enriched Table 1
The pivoted version orients characteristics as rows and splits (Train / Val / Test) as columns. Added fields: weight, height, ethnicity distribution (%), length of stay (LOS) summary, and additional vitals/labs if present. Continuous variables are reported as `mean (SD)`. Categorical distributions are reported as `% of split`. Missing variables are skipped automatically.

In [ ]:
# Multi-target baseline (logistic) for reference per target
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

baseline_results = {}
X_base = X_tr  # using training split from shared split
X_val_base = X_val

for tgt_name, label_col in TARGET_SPECS:
    y_tr_t = labels_df[label_col].astype(int).to_numpy()[idx_tr]
    y_val_t = labels_df[label_col].astype(int).to_numpy()[idx_val]
    pipe = Pipeline([
        ("imp", SimpleImputer(strategy='median')),
        ("sc", StandardScaler(with_mean=False)),
        ("lr", LogisticRegression(max_iter=500, class_weight='balanced', solver='liblinear'))
    ])
    pipe.fit(X_base, y_tr_t)
    val_proba = pipe.predict_proba(X_val_base)[:,1]
    auc_val = roc_auc_score(y_val_t, val_proba)
    baseline_results[tgt_name] = float(auc_val)
print('Baseline logistic validation AUCs per target:', baseline_results)

## 7. Model Training & Tuning
Hyperparameter optimization (Optuna) + final & calibration model training with isotonic calibration and F1‑optimal threshold selection on validation.

In [ ]:
# Multi-target tuning & training pipeline (updated to persist final_model + artifact paths)
import optuna, numpy as np, json, joblib
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from xgboost import XGBClassifier


# Hyperparameter search configuration
N_TRIALS_PER_TARGET = 1  # increase for better tuning
N_FOLDS = 5
MAX_ROUNDS = 400
SPEED_SAMPLE_MAX = 12000

def make_objective(X_train_full, y_train_full):
    def objective(trial: optuna.Trial):
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 7),
            'min_child_weight': trial.suggest_float('min_child_weight', 1.0, 8.0),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_lambda': trial.suggest_float('lambda', 1e-3, 5.0, log=True),
            'reg_alpha': trial.suggest_float('alpha', 1e-3, 2.0, log=True),
            'gamma': trial.suggest_float('gamma', 0.0, 4.0),
            'n_estimators': trial.suggest_int('n_estimators', 120, MAX_ROUNDS),
        }
        fold_aucs = []
        skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
        for fold,(tr_idx_local, va_idx_local) in enumerate(skf.split(X_train_full, y_train_full), 1):
            Xtr_f, Xva_f = X_train_full[tr_idx_local], X_train_full[va_idx_local]
            ytr_f, yva_f = y_train_full[tr_idx_local], y_train_full[va_idx_local]
            if Xtr_f.shape[0] > SPEED_SAMPLE_MAX:
                pos_idx = np.where(ytr_f==1)[0]
                neg_idx = np.where(ytr_f==0)[0]
                keep_pos = pos_idx
                remaining = SPEED_SAMPLE_MAX - len(keep_pos)
                if remaining < len(neg_idx):
                    keep_neg = np.random.default_rng(SEED+fold).choice(neg_idx, size=remaining, replace=False)
                else:
                    keep_neg = neg_idx
                keep = np.concatenate([keep_pos, keep_neg])
                np.random.default_rng(SEED+fold).shuffle(keep)
                Xtr_f = Xtr_f[keep]; ytr_f = ytr_f[keep]
            pos_rate_fold = ytr_f.mean(); spw = (1-pos_rate_fold)/max(pos_rate_fold,1e-6)
            model = XGBClassifier(objective='binary:logistic', tree_method='hist', scale_pos_weight=spw, eval_metric='auc', verbosity=0, **params)
            model.fit(Xtr_f, ytr_f, verbose=False)
            proba = model.predict_proba(Xva_f)[:,1]
            fold_auc = roc_auc_score(yva_f, proba)
            fold_aucs.append(fold_auc)
        mean_auc = float(np.mean(fold_aucs))
        trial.set_user_attr('fold_aucs', fold_aucs)
        trial.set_user_attr('cv_mean_auc', mean_auc)
        return mean_auc
    return objective

for tgt_name, label_col in TARGET_SPECS:
    print('\n==== Target:', tgt_name, '('+label_col+') ====')
    y_all = labels_df[label_col].astype(int).to_numpy()
    y_tr_t = y_all[idx_tr]; y_val_t = y_all[idx_val]; y_te_t = y_all[idx_te]
    pos_rate = y_tr_t.mean(); scale_pos_weight = (1-pos_rate)/max(pos_rate,1e-6)
    print(f'Pos rate train: {pos_rate:.4f} -> spw {scale_pos_weight:.2f}')
    sampler = optuna.samplers.TPESampler(seed=SEED)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    objective = make_objective(X_tr, y_tr_t)
    study.optimize(objective, n_trials=N_TRIALS_PER_TARGET, show_progress_bar=False)
    best_params = study.best_params.copy()
    print('Best CV mean AUC:', round(study.best_value,4))
    print('Best Params:', best_params)
    # Final model retrained on train + validation (for potential deployment / ensembling)
    X_tr_full_t = np.vstack([X_tr, X_val])
    y_tr_full_t = np.concatenate([y_tr_t, y_val_t])
    final_model = XGBClassifier(objective='binary:logistic', tree_method='hist', learning_rate=best_params['learning_rate'], n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], min_child_weight=best_params['min_child_weight'], subsample=best_params['subsample'], colsample_bytree=best_params['colsample_bytree'], reg_lambda=best_params['lambda'], reg_alpha=best_params['alpha'], gamma=best_params['gamma'], scale_pos_weight=scale_pos_weight, eval_metric='auc', verbosity=0)
    final_model.fit(X_tr_full_t, y_tr_full_t)
    # Calibration base model (train-only) to avoid leaking validation into both model weights and calibrator
    base_model = XGBClassifier(objective='binary:logistic', tree_method='hist', learning_rate=best_params['learning_rate'], n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], min_child_weight=best_params['min_child_weight'], subsample=best_params['subsample'], colsample_bytree=best_params['colsample_bytree'], reg_lambda=best_params['lambda'], reg_alpha=best_params['alpha'], gamma=best_params['gamma'], scale_pos_weight=scale_pos_weight, eval_metric='logloss', verbosity=0)
    base_model.fit(X_tr, y_tr_t)
    val_raw = base_model.predict_proba(X_val)[:,1]
    iso = IsotonicRegression(out_of_bounds='clip'); iso.fit(val_raw, y_val_t)
    val_cal = iso.transform(val_raw)
    # Threshold selection on calibrated validation probabilities (maximize F1)
    ths = np.linspace(0.01,0.9,300)
    best_thr = None; best_f1 = -1; best_val_metrics = None
    def _metrics_at(proba, y, thr):
        pred = (proba >= thr).astype(int)
        tp = ((pred==1)&(y==1)).sum(); fp = ((pred==1)&(y==0)).sum(); fn = ((pred==0)&(y==1)).sum()
        prec = tp/(tp+fp+1e-9); rec = tp/(tp+fn+1e-9)
        f1 = 2*prec*rec/(prec+rec+1e-9)
        return dict(precision=float(prec), recall=float(rec), f1=float(f1))
    for t in ths:
        m = _metrics_at(val_cal, y_val_t, t)
        if m['f1'] > best_f1:
            best_f1 = m['f1']; best_thr = float(t); best_val_metrics = m
    print('Selected threshold (validation calibrated):', best_thr, best_val_metrics)
    # Test evaluation using calibrated base model
    test_cal = iso.transform(base_model.predict_proba(X_te)[:,1])
    auc = roc_auc_score(y_te_t, test_cal)
    pr_auc = average_precision_score(y_te_t, test_cal)
    brier = brier_score_loss(y_te_t, test_cal)
    test_pred = (test_cal >= best_thr).astype(int)
    tp = ((test_pred==1)&(y_te_t==1)).sum(); fp = ((test_pred==1)&(y_te_t==0)).sum(); fn = ((test_pred==0)&(y_te_t==1)).sum()
    prec_test = tp/(tp+fp+1e-9); rec_test = tp/(tp+fn+1e-9)
    f1_test = 2*prec_test*rec_test/(prec_test+rec_test+1e-9)
    metrics = {'auc': float(auc), 'pr_auc': float(pr_auc), 'brier': float(brier), 'threshold': best_thr, 'f1_at_threshold': float(f1_test), 'precision_at_threshold': float(prec_test), 'recall_at_threshold': float(rec_test), 'validation_threshold_info': {**best_val_metrics, 'threshold': best_thr}, 'optuna_best_value_cv_mean_auc': float(study.best_value), 'optuna_best_params': best_params, 'train_rows': int(X_tr.shape[0]), 'val_rows': int(X_val.shape[0]), 'test_rows': int(X_te.shape[0])}
    metrics_all[tgt_name] = metrics
    print('Test metrics:', json.dumps(metrics, indent=2))
    # Persist artifacts (both final_model and calibration pipeline components)
    ARTIFACTS_DIR.mkdir(exist_ok=True)
    final_path = ARTIFACTS_DIR / f'model_full_{tgt_name}.joblib'  # trained on train+val
    base_path = ARTIFACTS_DIR / f'model_{tgt_name}.joblib'        # train-only (paired with isotonic)
    iso_path = ARTIFACTS_DIR / f'isotonic_{tgt_name}.joblib'
    joblib.dump(final_model, final_path)
    joblib.dump(base_model, base_path)
    joblib.dump(iso, iso_path)
    with open(ARTIFACTS_DIR / f'metrics_{tgt_name}.json','w') as f: json.dump(metrics, f, indent=2)
    with open(ARTIFACTS_DIR / f'threshold_{tgt_name}.txt','w') as f: f.write(str(best_thr))
    with open(ARTIFACTS_DIR / f'best_params_{tgt_name}.json','w') as f: json.dump(best_params, f, indent=2)
    print(f'Artifact paths ({tgt_name}):')
    for p in [final_path, base_path, iso_path, ARTIFACTS_DIR / f'metrics_{tgt_name}.json', ARTIFACTS_DIR / f'threshold_{tgt_name}.txt', ARTIFACTS_DIR / f'best_params_{tgt_name}.json']:
        print('  -', p.resolve())


## 8. Metrics Aggregation & Artifact Index
Persist per‑target metrics + consolidated index of models, calibrators, thresholds, and hyperparameters.

In [ ]:

        
# Combined metrics + artifact index
with open(ARTIFACTS_DIR / 'metrics_all.json','w') as f: json.dump(metrics_all, f, indent=2)
artifact_index = {}
for tgt_name,_ in TARGET_SPECS:
    artifact_index[tgt_name] = {
        'final_model': str((ARTIFACTS_DIR / f'model_full_{tgt_name}.joblib').resolve()),
        'base_model': str((ARTIFACTS_DIR / f'model_{tgt_name}.joblib').resolve()),
        'isotonic': str((ARTIFACTS_DIR / f'isotonic_{tgt_name}.joblib').resolve()),
        'metrics': str((ARTIFACTS_DIR / f'metrics_{tgt_name}.json').resolve()),
        'threshold': str((ARTIFACTS_DIR / f'threshold_{tgt_name}.txt').resolve()),
        'best_params': str((ARTIFACTS_DIR / f'best_params_{tgt_name}.json').resolve())
    }
with open(ARTIFACTS_DIR / 'artifact_index.json','w') as f: json.dump(artifact_index, f, indent=2)
print('All target metrics written ->', (ARTIFACTS_DIR / 'metrics_all.json').resolve())
print('Artifact index written ->', (ARTIFACTS_DIR / 'artifact_index.json').resolve())

In [ ]:
# Summarize saved artifacts for all targets
import json, os
artifact_summary = {}
for tgt_name, _ in TARGET_SPECS:
    tgt_files = [f for f in os.listdir(ARTIFACTS_DIR) if f.startswith(('model_'+tgt_name,'isotonic_'+tgt_name,'metrics_'+tgt_name,'threshold_'+tgt_name,'best_params_'+tgt_name))]
    artifact_summary[tgt_name] = tgt_files
print(json.dumps(artifact_summary, indent=2))

## 9. ROC & PR Curves (Calibrated)
Visual diagnostics for calibrated probability discrimination across targets.

In [ ]:
# ROC & PR curves (calibrated) for all targets
import json
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve, auc as sk_auc
from joblib import load as jobload

roc_data = {}
pr_data = {}
for tgt_name, label_col in TARGET_SPECS:
    model_path = ARTIFACTS_DIR / f'model_{tgt_name}.joblib'
    iso_path = ARTIFACTS_DIR / f'isotonic_{tgt_name}.joblib'
    if not (model_path.exists() and iso_path.exists()):
        print('Missing artifacts for', tgt_name)
        continue
    model = jobload(model_path)
    iso = jobload(iso_path)
    y_all = labels_df[label_col].astype(int).to_numpy()
    y_te_t = y_all[idx_te]
    raw = model.predict_proba(X_te)[:,1]
    cal = iso.transform(raw)
    fpr,tpr,_ = roc_curve(y_te_t, cal)
    prec,rec,_ = precision_recall_curve(y_te_t, cal)
    roc_auc = sk_auc(fpr,tpr)
    pr_auc = sk_auc(rec,prec)
    roc_data[tgt_name] = (fpr,tpr,roc_auc)
    pr_data[tgt_name] = (rec,prec,pr_auc)

fig, axes = plt.subplots(1,2, figsize=(12,5))
for tgt,(fpr,tpr,roc_auc) in roc_data.items():
    axes[0].plot(fpr,tpr,label=f"{tgt} AUC={roc_auc:.3f}")
axes[0].plot([0,1],[0,1],'--',color='grey'); axes[0].set_title('ROC (Calibrated)'); axes[0].legend(); axes[0].set_xlabel('FPR'); axes[0].set_ylabel('TPR')
for tgt,(rec,prec,pr_auc) in pr_data.items():
    axes[1].plot(rec,prec,label=f"{tgt} PR AUC={pr_auc:.3f}")
axes[1].set_title('PR (Calibrated)'); axes[1].legend(); axes[1].set_xlabel('Recall'); axes[1].set_ylabel('Precision')
plt.tight_layout(); plt.show()

## 10. Reliability (Calibration) Curves
Assess probability calibration pre/post isotonic across test set.

In [ ]:
# Calibration curves for all targets (using shared_inference unified loading)
from project.inference import get_model_and_calibrator, apply_calibration, _resolve_models_dir
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
import numpy as np

ARTIFACTS_DIR = _resolve_models_dir()

calibration_targets = []  # (tgt_name, y_true_test, raw_probs, calibrated_probs)
for tgt_name, label_col in TARGET_SPECS:
    try:
        model, calibrator = get_model_and_calibrator(tgt_name, models_dir=ARTIFACTS_DIR)
    except FileNotFoundError:
        print('Skip (missing artifacts):', tgt_name)
        continue
    y_all = labels_df[label_col].astype(int).to_numpy(); y_te_t = y_all[idx_te]
    raw = model.predict_proba(X_te)[:,1]
    cal = apply_calibration(raw, calibrator)
    calibration_targets.append((tgt_name, y_te_t, raw, cal))

if not calibration_targets:
    raise RuntimeError('No targets available for calibration plotting.')

fig, axes = plt.subplots(1, len(calibration_targets), figsize=(5*len(calibration_targets),4), sharey=True)
if len(calibration_targets) == 1:
    axes = [axes]

for ax,(tgt,y_true,raw,cal) in zip(axes, calibration_targets):
    fr_raw, mp_raw = calibration_curve(y_true, raw, n_bins=15, strategy='quantile')
    fr_cal, mp_cal = calibration_curve(y_true, cal, n_bins=15, strategy='quantile')
    ax.plot(mp_raw, fr_raw, 'o-', label='Raw', alpha=0.7)
    ax.plot(mp_cal, fr_cal, 'o-', label='Calibrated', alpha=0.7)
    ax.plot([0,1],[0,1], '--', color='gray')
    ax.set_title(f'Calibration: {tgt}')
    ax.set_xlabel('Predicted'); ax.set_ylabel('Observed')
    ax.legend()
plt.tight_layout(); plt.show()

# Report average absolute calibration shift per target
for tgt, y_true, raw, cal in calibration_targets:
    shift = float(np.mean(np.abs(raw - cal)))
    print(f"{tgt}: mean |raw-cal| = {shift:.5f}")

## 11. (Legacy) Manual Calibration Example
Illustrative manual isotonic calibration + threshold derivation (kept for reference; main pipeline already calibrates per‑target).

In [ ]:
# Manual isotonic calibration (sklearn XGBClassifier base, no DMatrix)
from xgboost import XGBClassifier
from sklearn.isotonic import IsotonicRegression
import numpy as np

# Base model: train-only (exclude validation for calibration fairness)
params = study.best_params.copy()
base_model = XGBClassifier(
    objective='binary:logistic', tree_method='hist',
    learning_rate=params['learning_rate'],
    n_estimators=params['n_estimators'],
    max_depth=params['max_depth'],
    min_child_weight=params['min_child_weight'],
    subsample=params['subsample'],
    colsample_bytree=params['colsample_bytree'],
    reg_lambda=params['lambda'],
    reg_alpha=params['alpha'],
    gamma=params['gamma'],
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    verbosity=0
)
base_model.fit(X_tr, y_tr)
val_proba_raw = base_model.predict_proba(X_val)[:,1]
iso = IsotonicRegression(out_of_bounds='clip')
iso.fit(val_proba_raw, y_val)
print('Isotonic calibration fitted on validation set.')

def predict_calibrated(X):
    return iso.transform(base_model.predict_proba(X)[:,1])

# Derive operating threshold on calibrated validation probabilities
val_cal = predict_calibrated(X_val)
ths = np.linspace(0.01,0.9,300)
threshold_info = None
for t in ths:
    m = metrics_at(val_cal, y_val, t)
    if (threshold_info is None) or (m['f1'] > threshold_info['f1']):
        threshold_info = {**m, 'threshold': float(t)}
print('Selected threshold (calibrated validation):', threshold_info)

## 12. Metrics Summary & Persistence
Create comparative summary tables (tuned vs baseline; full vs calibrated) and persist to artifacts directory.

In [ ]:
# Unified summary table: tuned vs baseline metrics per target
import json, pandas as pd
rows = []
for tgt_name, _ in TARGET_SPECS:
    metrics_path = ARTIFACTS_DIR / f'metrics_{tgt_name}.json'
    if not metrics_path.exists():
        continue
    with open(metrics_path) as f:
        m = json.load(f)
    rows.append({
        'target': tgt_name,
        'auc': m['auc'],
        'pr_auc': m['pr_auc'],
        'brier': m['brier'],
        'threshold': m['threshold'],
        'f1_at_threshold': m['f1_at_threshold'],
        'precision_at_threshold': m['precision_at_threshold'],
        'recall_at_threshold': m['recall_at_threshold'],
        'baseline_val_auc': baseline_results.get(tgt_name)
    })
summary_df = pd.DataFrame(rows)
print(summary_df.sort_values('target'))
summary_df.to_csv(ARTIFACTS_DIR / 'metrics_summary.csv', index=False)
print('Wrote metrics_summary.csv')

In [ ]:
# Compare full (train+val) vs base calibrated models on test
from joblib import load as jobload
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

comparison_rows = []
missing_any = False
for tgt_name, label_col in TARGET_SPECS:
    y_all = labels_df[label_col].astype(int).to_numpy()
    y_te_t = y_all[idx_te]
    full_path = ARTIFACTS_DIR / f'model_full_{tgt_name}.joblib'
    base_path = ARTIFACTS_DIR / f'model_{tgt_name}.joblib'
    iso_path = ARTIFACTS_DIR / f'isotonic_{tgt_name}.joblib'
    if not (full_path.exists() and base_path.exists() and iso_path.exists()):
        print('Skipping', tgt_name, 'missing one of required artifacts')
        missing_any = True
        continue
    full_model = jobload(full_path)
    base_model = jobload(base_path)
    iso = jobload(iso_path)
    raw_full = full_model.predict_proba(X_te)[:,1]
    raw_base = base_model.predict_proba(X_te)[:,1]
    cal_base = iso.transform(raw_base)
    row = {
        'target': tgt_name,
        'full_raw_auc': roc_auc_score(y_te_t, raw_full),
        'full_raw_pr_auc': average_precision_score(y_te_t, raw_full),
        'base_raw_auc': roc_auc_score(y_te_t, raw_base),
        'base_cal_auc': roc_auc_score(y_te_t, cal_base),
        'base_cal_pr_auc': average_precision_score(y_te_t, cal_base),
        'base_cal_brier': brier_score_loss(y_te_t, cal_base)
    }
    comparison_rows.append(row)

comparison_df = pd.DataFrame(comparison_rows)
if len(comparison_df):
    display(comparison_df.sort_values('target'))
    comparison_df.to_csv(ARTIFACTS_DIR / 'full_vs_calibrated_comparison.csv', index=False)
    print('Wrote full_vs_calibrated_comparison.csv')
else:
    print('No comparison rows generated.')

## 13. Inference & Enriched Metrics
Merge primary metrics with comparison outputs for downstream reporting; quick sanity preview of artifacts.

In [ ]:
# Merge comparison with metrics summary (enriched)
import pandas as pd, json
summary_path = ARTIFACTS_DIR / 'metrics_summary.csv'
comp_path = ARTIFACTS_DIR / 'full_vs_calibrated_comparison.csv'
if summary_path.exists() and comp_path.exists():
    summary_df = pd.read_csv(summary_path)
    comp_df = pd.read_csv(comp_path)
    merged = summary_df.merge(comp_df, on='target', how='left')
    merged.to_csv(ARTIFACTS_DIR / 'metrics_summary_enriched.csv', index=False)
    print('Wrote metrics_summary_enriched.csv')
    display(merged.sort_values('target'))
else:
    print('Missing one of summary or comparison CSV; skip enrichment.')

In [ ]:
# AUC Difference Significance (Bootstrap between full_raw and base_cal)
import numpy as np, pandas as pd
from joblib import load as jobload
from sklearn.metrics import roc_auc_score

RESULTS = []
N_BOOT = 2000  # increase for tighter CI (runtime ~ O(N_BOOT))
RNG = np.random.default_rng(42)

for tgt_name, label_col in TARGET_SPECS:
    full_path = ARTIFACTS_DIR / f'model_full_{tgt_name}.joblib'
    base_path = ARTIFACTS_DIR / f'model_{tgt_name}.joblib'
    iso_path = ARTIFACTS_DIR / f'isotonic_{tgt_name}.joblib'
    if not (full_path.exists() and base_path.exists() and iso_path.exists()):
        print('Skip significance for', tgt_name, '(missing artifacts)')
        continue
    y_all = labels_df[label_col].astype(int).to_numpy()
    y_te_t = y_all[idx_te]
    full_model = jobload(full_path)
    base_model = jobload(base_path)
    iso = jobload(iso_path)
    raw_full = full_model.predict_proba(X_te)[:,1]
    cal_base = iso.transform(base_model.predict_proba(X_te)[:,1])
    auc_full = roc_auc_score(y_te_t, raw_full)
    auc_cal = roc_auc_score(y_te_t, cal_base)
    diff = auc_full - auc_cal
    n = len(y_te_t)
    # Bootstrap
    diffs = np.empty(N_BOOT)
    for b in range(N_BOOT):
        idx = RNG.integers(0, n, size=n)
        y_b = y_te_t[idx]
        rf_b = raw_full[idx]
        cb_b = cal_base[idx]
        try:
            diffs[b] = roc_auc_score(y_b, rf_b) - roc_auc_score(y_b, cb_b)
        except ValueError:
            # In rare case bootstrap sample has only one class
            diffs[b] = np.nan
    diffs = diffs[~np.isnan(diffs)]
    if len(diffs) < N_BOOT * 0.9:
        print('Warning: many degenerate bootstrap samples for', tgt_name)
    lower, upper = np.percentile(diffs, [2.5, 97.5])
    # Two-sided p-value: proportion of bootstrap diffs with opposite sign or more extreme
    if diff >= 0:
        p = (np.sum(diffs <= 0) + 1) / (len(diffs) + 1)
    else:
        p = (np.sum(diffs >= 0) + 1) / (len(diffs) + 1)
    RESULTS.append({
        'target': tgt_name,
        'auc_full_raw': auc_full,
        'auc_base_cal': auc_cal,
        'diff_full_minus_cal': diff,
        'ci_95_lower': lower,
        'ci_95_upper': upper,
        'approx_p_two_sided': p * 2 if p * 2 <= 1 else 1.0,
        'n_test': n,
        'n_boot_effective': int(len(diffs))
    })

if RESULTS:
    sig_df = pd.DataFrame(RESULTS).sort_values('target')
    display(sig_df)
    sig_df.to_csv(ARTIFACTS_DIR / 'auc_diff_significance.csv', index=False)
    print('Wrote auc_diff_significance.csv')
else:
    print('No targets processed for significance analysis.')

## 14. SHAP Feature Attribution
Compute post‑hoc feature attributions using SHAP (TreeExplainer) on the trained base XGBoost models (the ones paired with isotonic calibration). Rationale: calibrated probabilities are a monotonic transformation of raw model scores; isotonic preserves rank order locally, so SHAP on the raw base model approximates contributions to the calibrated output.

Methodology:
- Use `TreeExplainer` for efficiency on gradient boosted trees.
- Sample up to 4,000 training instances (stratified) as background to reduce memory while preserving class signal.
- Compute SHAP values on the held‑out test set only (avoid leakage and align with evaluation metrics).
- Aggregate mean |SHAP| per feature for global importance; persist full matrix for potential future local explanations.
- Save: CSV (importance ordering), raw `.npy` dense SHAP array, and PNG summary bar plot per target.

Notes:
- For very wide feature spaces, you can optionally sparsify or threshold low‑variance columns earlier to reduce runtime.
- If features contain derived aggregations (e.g., max/min/mean vitals), interpret top drivers in clinical context before decision support use.
- Future extension: compute per‑demographic group mean |SHAP| to explore feature usage disparity alongside performance fairness.


In [ ]:
# SHAP computation for each target (base model perspective)
import numpy as np, pandas as pd, json, os, gc
from joblib import load as jobload
import shap
import matplotlib.pyplot as plt

ARTIFACTS_DIR.mkdir(exist_ok=True)

X_test = X_te  # test matrix (numpy array)
feature_names = list(feature_df.columns)

# Background sample (stratified on primary target) to cap complexity
MAX_BACKGROUND = 4000
background_indices = idx_tr
y_primary = labels_df[TARGET_SPECS[0][1]].astype(int).to_numpy()
train_labels_primary = y_primary[idx_tr]

if len(background_indices) > MAX_BACKGROUND:
    df_bg = pd.DataFrame({'idx': background_indices, 'y': train_labels_primary})
    pos = df_bg[df_bg.y==1]; neg = df_bg[df_bg.y==0]
    # Keep min(all positives, half budget)
    target_pos = min(len(pos), MAX_BACKGROUND//2)
    pos_sample = pos.sample(n=target_pos, random_state=SEED) if len(pos) > target_pos else pos
    remaining = MAX_BACKGROUND - len(pos_sample)
    neg_sample = neg.sample(n=min(remaining, len(neg)), random_state=SEED)
    bg_sample = pd.concat([pos_sample, neg_sample]).sample(frac=1, random_state=SEED)
    background_indices = bg_sample['idx'].to_numpy()

X_background = X[background_indices]
print(f'SHAP background size: {X_background.shape[0]} (from train)')

shap_artifacts = []

for tgt_name, label_col in TARGET_SPECS:
    base_path = ARTIFACTS_DIR / f'model_{tgt_name}.joblib'
    if not base_path.exists():
        print(f'Skip SHAP for {tgt_name}: missing base model')
        continue
    print(f'Computing SHAP for target: {tgt_name}')
    try:
        model = jobload(base_path)
        explainer = shap.TreeExplainer(model, feature_perturbation='tree_path_dependent')
        shap_values = explainer.shap_values(X_test)
    except Exception as e:
        print(f'Failed SHAP for {tgt_name}: {e}')
        continue

    # Handle potential list (multi-class); our case is binary so expect 2-d
    if isinstance(shap_values, list):
        # If shap returns list (e.g., per class), aggregate class 1 if available
        shap_values = shap_values[1] if len(shap_values) > 1 else shap_values[0]

    abs_mean = np.abs(shap_values).mean(axis=0)
    importance_df = pd.DataFrame({'feature': feature_names, 'mean_abs_shap': abs_mean}).sort_values('mean_abs_shap', ascending=False)
    csv_path = ARTIFACTS_DIR / f'shap_feature_importance_{tgt_name}.csv'
    npy_path = ARTIFACTS_DIR / f'shap_values_{tgt_name}.npy'
    importance_df.to_csv(csv_path, index=False)
    np.save(npy_path, shap_values)

    # Horizontal bar plot (top 25)
    top_k = min(25, len(importance_df))
    top_imp = importance_df.head(top_k)
    plt.figure(figsize=(8, max(5, top_k*0.25)))
    plt.barh(top_imp['feature'][::-1], top_imp['mean_abs_shap'][::-1], color='steelblue')
    plt.xlabel('Mean |SHAP| (test)')
    plt.title(f'SHAP Global Importance (Top {top_k}) - {tgt_name}')
    plt.tight_layout()
    png_bar = ARTIFACTS_DIR / f'shap_summary_{tgt_name}.png'
    plt.savefig(png_bar, dpi=160)
    plt.show()

    # Beeswarm plot
    png_bee = None
    try:
        shap.summary_plot(shap_values, features=X_test, feature_names=feature_names, max_display=25, show=False)
        plt.tight_layout()
        png_bee = ARTIFACTS_DIR / f'shap_beeswarm_{tgt_name}.png'
        plt.savefig(png_bee, dpi=160)
        plt.show()
    except Exception as e:
        print(f'Beeswarm failed ({tgt_name}): {e}')

    shap_artifacts.append({
        'target': tgt_name,
        'importance_csv': str(csv_path),
        'values_npy': str(npy_path),
        'bar_png': str(png_bar),
        'beeswarm_png': str(png_bee) if png_bee else None
    })

    # Free memory early for large matrices
    del shap_values; gc.collect()

print('SHAP artifacts written:')
print(json.dumps(shap_artifacts, indent=2))